In [6]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from scipy.stats import linregress

In [41]:
try:
    import pymannkendall as mk
except Exception:
    mk = None
from statsmodels.stats.multitest import multipletests
data_path = Path("E:/Geospatial_Data_Science/Portfolio/Ohio River Project/data/raw/Ohio_river.csv")
result_path = Path("E:/Geospatial_Data_Science/Portfolio/Ohio River Project/data/processed")
out_dir = Path("E:/Geospatial_Data_Science/Portfolio/Ohio River Project/outputs")
out_dir.mkdir(exist_ok=True)


In [10]:
# loading data 
df = pd.read_csv(data_path)
df.head()

,River,Confluence Mile Point,Bimonthly SiteName,Latitude,Longitude,Date,Parameter (Units),PublicData,Parameter Long Name
0,Ohio,15.2,South Heights,40.57000,-80.229722,7/1/1976,Flow (cfs),13000,Flow
1,Beaver,25.4,Beaver Falls,40.76333,-80.315280,7/1/1976,Flow (cfs),1900,Flow
2,Beaver,25.4,Beaver Falls,40.76333,-80.315280,7/1/1976,NH3-N (mg/L),0.52,"Nitrogen, Ammonia"
3,Beaver,25.4,Beaver Falls,40.76333,-80.315280,7/1/1976,TKN (mg/L),1.2,"Nitrogen, Kjeldahl, Total"
4,Beaver,25.4,Beaver Falls,40.76333,-80.315280,7/1/1976,NO2-NO3-N (mg/L),1.9,"Nitrogen, NO2 plus NO3"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261796 entries, 0 to 261795
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   River                  261796 non-null  object 
 1   Confluence Mile Point  261796 non-null  float64
 2   Bimonthly SiteName     261796 non-null  object 
 3   Latitude               261796 non-null  float64
 4   Longitude              261796 non-null  float64
 5   Date                   261796 non-null  object 
 6   Parameter (Units)      261796 non-null  object 
 7   PublicData             261796 non-null  object 
 8   Parameter Long Name    261796 non-null  object 
dtypes: float64(3), object(6)
memory usage: 18.0+ MB


## Data Cleaning 
As part of data cleaning, let's clarify our coulum names. We will create a dictionary called col_rename, and replace the old column names with our prefered names for colunms. 


In [27]:
# creating a dictionary with new column headings
col_rename = {
    'Parameter (Units)': 'parameter_units',
    'PublicData': 'value_raw',
    'Bimonthly SiteName': 'site_name',
    'Confluence Mile Point': 'mile_point',
    'Parameter Long Name': 'parameter_long'
}
# looping through each keys and checking if it matches with our dataframe and return a new dictionary which then is used to rename columns headings
new_dict = {}
for k in col_rename:
    if k in df.columns:
        new_dict[k]= col_rename[k]

df2 = df.rename(columns=new_dict)
# Lets make sure the Date is datetime
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')

# To parse parameter and units
def split_param_units(s):
    if pd.isna(s): return (np.nan, np.nan)
    m = re.match(r'(.+?)\s*\((.+)\)\s*$', str(s).strip())
    if m:
        return m.group(1).strip(), m.group(2).strip()
    else:
        return str(s).strip(), np.nan

df2[['parameter','units']] = df2['parameter_units'].apply(lambda x: pd.Series(split_param_units(x)))

# standardize column names
expected_cols = ['River','mile_point','site_name','Latitude','Longitude','Date','parameter','units','value_raw','parameter_long']
for c in expected_cols:
    if c not in df2.columns:
        print("Warning: missing column", c)

df2 = df2.rename(columns={'Latitude':'latitude','Longitude':'longitude'})
df2 = df2[['River','mile_point','site_name','latitude','longitude','Date','parameter','units','value_raw','parameter_long']]
df2.head()


,River,mile_point,site_name,latitude,longitude,Date,parameter,units,value_raw,parameter_long
0,Ohio,15.2,South Heights,40.57000,-80.229722,1976-07-01,Flow,cfs,13000,Flow
1,Beaver,25.4,Beaver Falls,40.76333,-80.315280,1976-07-01,Flow,cfs,1900,Flow
2,Beaver,25.4,Beaver Falls,40.76333,-80.315280,1976-07-01,NH3-N,mg/L,0.52,"Nitrogen, Ammonia"
3,Beaver,25.4,Beaver Falls,40.76333,-80.315280,1976-07-01,TKN,mg/L,1.2,"Nitrogen, Kjeldahl, Total"
4,Beaver,25.4,Beaver Falls,40.76333,-80.315280,1976-07-01,NO2-NO3-N,mg/L,1.9,"Nitrogen, NO2 plus NO3"


In [28]:
# To handle missing values
def parse_value(s):
    if pd.isna(s):
        return (np.nan, False, np.nan)
    s_str = str(s).strip()
 # handle <, <= nondetects
    m = re.match(r'^\s*[<≤]\s*([0-9\.\-eE]+)\s*$', s_str)
    if m:
        dl = float(m.group(1))
        return (np.nan, True, dl)
    # handle qualifiers like "<=10"
    m2 = re.match(r'^[\<≤]\s*([0-9\.\-eE]+)', s_str)
    if m2:
        dl = float(m2.group(1))
        return (np.nan, True, dl)
    # or try numeric value
    try:
        val = float(s_str)
        return (val, False, np.nan)
    except:
        # maybe comma decimal, remove commas
        s2 = s_str.replace(',','')
        try:
            val = float(s2)
            return (val, False, np.nan)
        except:
            return (np.nan, False, np.nan)

parsed = df2['value_raw'].apply(lambda x: pd.Series(parse_value(x), index=['value','is_nd','dl']))
df2 = pd.concat([df, parsed], axis=1)

df2

,River,Confluence Mile Point,Bimonthly SiteName,Latitude,Longitude,Date,Parameter (Units),PublicData,Parameter Long Name,value,is_nd,dl
0,Ohio,15.2,South Heights,40.570000,-80.229722,1976-07-01,Flow (cfs),13000,Flow,13000.00,False,NaN
1,Beaver,25.4,Beaver Falls,40.763330,-80.315280,1976-07-01,Flow (cfs),1900,Flow,1900.00,False,NaN
2,Beaver,25.4,Beaver Falls,40.763330,-80.315280,1976-07-01,NH3-N (mg/L),0.52,"Nitrogen, Ammonia",0.52,False,NaN
3,Beaver,25.4,Beaver Falls,40.763330,-80.315280,1976-07-01,TKN (mg/L),1.2,"Nitrogen, Kjeldahl, Total",1.20,False,NaN
4,Beaver,25.4,Beaver Falls,40.763330,-80.315280,1976-07-01,NO2-NO3-N (mg/L),1.9,"Nitrogen, NO2 plus NO3",1.90,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
261791,Ohio,436.2,Meldahl,38.796394,-84.168965,2024-11-25,SO4 (mg/L),89.3,Sulfate,89.30,False,NaN
261792,Ohio,436.2,Meldahl,38.796394,-84.168965,2024-11-25,TDS (mg/L),268,Total Dissolved Solids,268.00,False,NaN
261793,Ohio,436.2,Meldahl,38.796394,-84.168965,2024-11-25,Hardness (mg/L),135,Total Hardness,135.00,False,NaN
261794,Ohio,436.2,Meldahl,38.796394,-84.168965,2024-11-25,TOC (mg/L),3,Total Organic Carbon,3.00,False,NaN


In [31]:
# Keep only NH3-N 
nh3 = df2[df2['Parameter (Units)'] == 'NH3-N (mg/L)'].copy()

# Drop rows with no numeric values
nh3 = nh3.dropna(subset=['value'])

print(nh3[['Date','Latitude','Longitude','value']].head())
print("Years covered:", nh3['Date'].min(), "to", nh3['Date'].max())


          Date   Latitude  Longitude  value
2   1976-07-01  40.763330 -80.315280   0.52
21  1976-07-06  40.410000 -79.954170   0.70
49  1976-07-08  39.360845 -81.319461   0.25
92  1976-07-12  39.119598 -81.742718   0.85
115 1976-07-12  38.913611 -82.127500   0.30
Years covered: 1976-07-01 00:00:00 to 2024-11-25 00:00:00


In [32]:
nh3['Year'] = nh3['Date'].dt.year
nh3['Decade'] = (nh3['Year'] // 10) * 10


In [36]:
# Compute yearly averages per station
nh3['Yearly_Mean'] = (
    nh3.groupby(['Latitude','Longitude','Year'])['value']
    .transform('mean')
)

nh3['Yearly_SampleCount'] = (
    nh3.groupby(['Latitude','Longitude','Year'])['value']
    .transform('count')
)


In [38]:
#  Group and calculate yearly summarie
yearly_summary = (
    nh3.groupby(['Latitude','Longitude','Year'])
    .agg(mean_value=('value','mean'),
         n_samples=('value','count'))
    .reset_index()
)

#  Merge back into main NH3 dataframe
nh3 = nh3.merge(yearly_summary, on=['Latitude','Longitude','Year'], how='left')


In [39]:
nh3.head(15)


,River,Confluence Mile Point,Bimonthly SiteName,Latitude,Longitude,Date,Parameter (Units),PublicData,Parameter Long Name,value,is_nd,dl,Year,Decade,Yearly_Mean,Yearly_SampleCount,mean_value,n_samples
0,Beaver,25.4,Beaver Falls,40.763330,-80.315280,1976-07-01,NH3-N (mg/L),0.52,"Nitrogen, Ammonia",0.52,False,NaN,1976,1970,0.818000,15,0.818000,15
1,Monongahela,0.0,South Pittsburgh,40.410000,-79.954170,1976-07-06,NH3-N (mg/L),0.7,"Nitrogen, Ammonia",0.70,False,NaN,1976,1970,0.479167,12,0.479167,12
2,Ohio,161.8,Willow Island,39.360845,-81.319461,1976-07-08,NH3-N (mg/L),0.25,"Nitrogen, Ammonia",0.25,False,NaN,1976,1970,0.474286,14,0.474286,14
3,Ohio,203.9,Belleville,39.119598,-81.742718,1976-07-12,NH3-N (mg/L),0.85,"Nitrogen, Ammonia",0.85,False,NaN,1976,1970,0.486154,13,0.486154,13
4,Ohio,260.0,Addison,38.913611,-82.127500,1976-07-12,NH3-N (mg/L),0.3,"Nitrogen, Ammonia",0.30,False,NaN,1976,1970,0.374167,12,0.374167,12
5,Ohio,279.2,R.C. Byrd,38.682426,-82.187935,1976-07-12,NH3-N (mg/L),0.3,"Nitrogen, Ammonia",0.30,False,NaN,1976,1970,0.362857,14,0.362857,14
6,Ohio,15.2,South Heights,40.570000,-80.229722,1976-07-14,NH3-N (mg/L),0.3,"Nitrogen, Ammonia",0.30,False,NaN,1976,1970,0.564286,14,0.564286,14
7,Beaver,25.4,Beaver Falls,40.763330,-80.315280,1976-07-14,NH3-N (mg/L),0.5,"Nitrogen, Ammonia",0.50,False,NaN,1976,1970,0.818000,15,0.818000,15
8,Ohio,40.2,East Liverpool,40.638889,-80.520833,1976-07-14,NH3-N (mg/L),0.5,"Nitrogen, Ammonia",0.50,False,NaN,1976,1970,0.658462,13,0.658462,13
9,Monongahela,0.0,South Pittsburgh,40.410000,-79.954170,1976-07-15,NH3-N (mg/L),0.35,"Nitrogen, Ammonia",0.35,False,NaN,1976,1970,0.479167,12,0.479167,12


In [43]:
# Save for ArcGISPro
nh3.to_csv(result_path / "NH3_cleaned_data.csv", index=False)
